<span style="color: green; font-size: 40px; font-weight: bold;">Lab 4 (Classificação Binária Probabilística) </span>

<br> <br>

# Prevendo Se Uma Mensagem de Texto é Spam

<br>

### Contexto

Neste projeto, abordaremos um problema comum em sistemas de comunicação digital: a **detecção de mensagens de texto indesejadas, conhecidas como spam**. Com o aumento do uso de mensagens eletrônicas, a capacidade de distinguir automaticamente entre mensagens legítimas e spam se tornou essencial para manter a integridade e a usabilidade dos serviços de mensagens. O objetivo deste projeto é desenvolver um modelo preditivo que, baseado no conteúdo textual das mensagens, consiga identificar se uma mensagem é spam ou não. Utilizaremos técnicas de Machine Learning para criar um modelo de classificação binária probabilística, capaz de fornecer uma previsão acompanhada de uma estimativa de probabilidade.

<br>

### Objetivo

O objetivo deste projeto é **construir um modelo de Machine Learning capaz de prever se uma mensagem de texto é spam**. O modelo será treinado utilizando dados históricos de mensagens rotuladas como "spam" ou "ham" (não spam), permitindo que ele faça previsões sobre novas mensagens com base em padrões aprendidos.

<br>

### Pergunta de Negócio Principal

> "Como podemos prever se uma mensagem de texto é spam utilizando seu conteúdo textual?"

<br>

### Entregável

O entregável deste projeto será um **modelo de Machine Learning treinado para identificar mensagens de texto como spam ou não**. O modelo será capaz de classificar novas mensagens e fornecer a probabilidade associada a cada previsão. O processo de desenvolvimento incluirá a preparação dos dados, a seleção de features relevantes, o treinamento do modelo e a avaliação de seu desempenho.

<br>

### Sobre o Conjunto de Dados

Os dados utilizados neste projeto contêm uma coleção de mensagens de texto classificadas manualmente como "spam" ou "ham". Cada entrada do conjunto de dados inclui o texto da mensagem e sua respectiva classificação. Utilizaremos esse conjunto para treinar e validar nosso modelo.

<br>
<table border="2">
  <tr>
    <th style="text-align: center; font-size: 16px;">Nome da Coluna</th>
    <th style="text-align: center; font-size: 16px;">Tipo de Dado</th>
    <th style="text-align: center; font-size: 16px;">Descrição</th>
  </tr>
  <tr>
    <td>label</td>
    <td>string</td>
    <td>Classificação da mensagem (ham para não spam, spam para mensagens indesejadas).</td>
  </tr>
  <tr>
    <td>message</td>
    <td>string</td>
    <td>Conteúdo textual da mensagem.</td>
  </tr>
</table>

<br><br><br>

# Importando Pacotes

In [ ]:
# Importa o findspark e inicializa
import findspark
findspark.init()

# Imports
import numpy as np
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession
from pyspark.ml import Pipeline
from pyspark.ml.feature import IDF, HashingTF, Tokenizer
from pyspark.ml.classification import NaiveBayes, NaiveBayesModel
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

<br> <br>

# <span style="color: green; font-size: 38px; font-weight: bold;">Preparando o Ambiente Spark</span>

In [ ]:
# Definindo semente aleatória (seed) para reprodutibilidade do notebook
rnd_seed = 23
np.random.seed = rnd_seed
np.random.set_state = rnd_seed

# Se houver uma sessão Spark ativa, encerre-a
if 'sc' in globals():
    sc.stop()

if 'spark' in globals():
    spark.stop()


# Criando o Spark Context
conf = SparkConf().setAppName("Lab4") \
                  .set("spark.ui.showConsoleProgress", "false") \
                  .set("spark.executor.heartbeatInterval", "20s") \
                  .set("spark.eventLog.enabled", "false") \
                  .set("spark.sql.shuffle.partitions", "2") \
                  .set("spark.sql.debug.maxToStringFields", "100") \
                  .set("spark.executor.memory", "4g") \
                  .set("spark.driver.memory", "4g") \
                  .set("spark.driver.maxResultSize", "2g")  # Configuração adicional para limitar o tamanho do resultado

# Criar o Spark Context e a Spark Session
sc = SparkContext(conf=conf)
spSession = SparkSession.builder.config(conf=conf).getOrCreate()

# Ajustar o nível de log para ERROR
sc.setLogLevel("ERROR")

# Configurar log4j para suprimir avisos (deixar como comentário e volta ao normal)
log4j_logger = sc._jvm.org.apache.log4j
log4j_logger.LogManager.getLogger("org").setLevel(log4j_logger.Level.ERROR)
log4j_logger.LogManager.getLogger("akka").setLevel(log4j_logger.Level.ERROR)

# Visualizar o objeto spark_session
spSession

<br><br>

# <span style="color: green; font-size: 38px; font-weight: bold;">Carregando os Dados</span>

- Os dados serão carregados a partir de um arquivo CSV e gerados como um RDD (Resilient Distributed Dataset) no Apache Spark. O RDD é uma estrutura de dados distribuída que permite o processamento paralelo em um cluster, otimizando a performance.

In [ ]:
# Carregando os dados e gerando um RDD
spamRDD = sc.textFile("Lab/dados/dataset3.csv")

# Tipo
print(type(spamRDD), '\n')

# Colocando o RDD em cache. Esse processo otimiza a performance
print(spamRDD.cache(), '\n')

# Número de registros
print(spamRDD.count(), '\n')

In [ ]:
# Visualizando as primeiras linhas
print(spamRDD.take(5))

In [ ]:
## Visualizando primeiras 4 linhas com Pandas (Apenas para visualização)

import pandas as pd

# Obter as primeiras 5 linhas do RDD
linhas = spamRDD.take(5)

# Dividir as linhas em colunas utilizando o ponto-e-vírgula como delimitador
colunas = linhas[0].split(";")
dados_formatados = [linha.split(";") for linha in linhas[1:]]

# Criar um DataFrame Pandas com as colunas e os dados formatados
df = pd.DataFrame(dados_formatados, columns=colunas)

# Mostrar o DataFrame
display(df)

<br><br><br>

# <span style="color: green; font-size: 38px; font-weight: bold;"> Análise Exploratória Inicial dos Dados </span>

<br>

### Criação de Função Para Análise Inicial

In [ ]:
import pandas as pd
import re

def funcao_analise_inicial(df):
    # Configurar Pandas para exibir todas as linhas
    pd.set_option('display.max_rows', None)

    # Informações do DataFrame
    print('\n\n INFO \n\n')
    df.info()
    print('\n\n ------------------------------------------------------------------------------------------ \n\n')

    # Verifica se há valores ausentes e duplicados
    valores_ausentes = df.isna().sum().sum() > 0
    valores_duplicados = df.duplicated().sum() > 0

    # Nomes das variáveis com valores ausentes
    variaveis_ausentes = df.columns[df.isna().any()].tolist()

    # Número de linhas duplicadas
    num_linhas_duplicadas = df.duplicated().sum()

    # Porcentagem de linhas duplicadas
    porcentagem_linhas_duplicadas = (num_linhas_duplicadas / len(df)) * 100

    # Exibe o resultado sobre valores ausentes e duplicados
    print("\n\nExistem valores ausentes:", valores_ausentes)
    if valores_ausentes:
        print("\nVariáveis com valores ausentes:", variaveis_ausentes)
    else:
        print("\nNenhuma variável possui valores ausentes.")

    print("\n\nExistem valores duplicados:", valores_duplicados)
    if valores_duplicados:
        print("\nNúmero de Linhas Duplicadas:", num_linhas_duplicadas)
        print("\nPorcentagem de Linhas Duplicadas: {:.2f}%".format(porcentagem_linhas_duplicadas))
    else:
        print("\nNenhuma variável possui valores duplicados.")
    
    # Verificação de caracteres especiais
    caracteres_especiais = re.compile('[@_!#$%^&*<>()?/\\|}{~:]')   # nenhum caracter removido
    colunas_com_caracteres_especiais = {}

    for coluna in df.columns:
        if df[coluna].dtype == 'object':  # Verifica apenas colunas de texto
            contem_caracteres_especiais = df[coluna].apply(lambda x: bool(caracteres_especiais.search(x) if isinstance(x, str) else False)).any()
            if contem_caracteres_especiais:
                indices_com_caracteres_especiais = df[coluna][df[coluna].apply(lambda x: bool(caracteres_especiais.search(x) if isinstance(x, str) else False))].index.tolist()
                colunas_com_caracteres_especiais[coluna] = indices_com_caracteres_especiais

    # Exibe o resultado sobre caracteres especiais
    print("\n\nExistem caracteres especiais nas colunas:", bool(colunas_com_caracteres_especiais))
    if colunas_com_caracteres_especiais:
        print("\nColunas com caracteres especiais e os índices:")
        for coluna, indices in colunas_com_caracteres_especiais.items():
            print(f"\n Coluna [ {coluna} ]: Índices com caracteres especiais {indices}")
    else:
        print("\nNenhuma coluna possui caracteres especiais.")

print('A função foi criada com sucesso.')

<br>

### Transformando dados carregados em RDD para dataframe do Pandas (apenas para Análise Inicial)

- Vamos realizar análise exploratória através da função acima. RDDs são ótimos para processamento, mas ruins para exploração, então converteremos o RDD para DataFrame Spark e então para DataFrame Pandas (**não é possível converter diretamente objeto RDD para objeto Pandas**).

In [ ]:
# Converte RDD para DataFrame Spark, corrigindo a separação de colunas
header = spamRDD.first().replace('"', '').split(';')
df_spark_com_cabecalho = spamRDD.map(lambda x: x.replace('"', '').split(';')).toDF(header)

# Remover a primeira linha do DataFrame, que é o cabeçalho
df_spark_com_cabecalho = df_spark_com_cabecalho.filter(df_spark_com_cabecalho[header[0]] != header[0])

# Verificar o tipo do objeto
print(type(df_spark_com_cabecalho), '\n')

# Converte DataFrame Spark para DataFrame Pandas
df_pandas = df_spark_com_cabecalho.toPandas()

# Visualizando as primeiras linhas do DataFrame Pandas
display(df_pandas.head())

<br>

### Visualizando Função para Análise Inicial

In [ ]:
funcao_analise_inicial(df_pandas)

### Resumo

- 

<br> <br> <br>
